Enunciado: Desarrollar un sistema de evento discreto para una agencia de transporte utilizando Simpy. Caso practico:

La empresa de transporte Cuenca desea saber cual es su ganancia si aumenta su flota a 7 buses teniendo en cuenta los siguientes aspectos:

    Los buses cubre la ruta de Norte Sur de la ciudad.
    Trabajan de 6:00 a 22:00 con un descanso de 1 hora en el almuerzo
    Cada ruta se demora 1:30 cada uno de los buses.
    Tiene un descanso de 30 minutos al finalizar el recorrido
    La capacidad máxima es de 65 personas
    Tienen 20 paradas dentro de la ruta.
        En cada parada se suben de 3 – 7 pasajeros .
        Se bajan de 5 – 7 pasajeros
        EL 70% son personas que pagan el pasaje completo
        El 30% estudiantes y tercera edad
        Los valores corresponde 0.30 centavos
        La ultima parada todos se bajan sin excepción.



In [1]:
!pip install simpy

In [2]:
import simpy
import random
import matplotlib.pyplot as plt
from scipy import stats 
import numpy as np
import math
import collections

In [3]:
cantidadBuses=7 #buses
tiempoSimulacion=960 #minutos
horaAlmuerzo=60 #minutos
tiempoRuta=90 #minutos
descansoFinalizarRecorriedo=30 #minutos
cantidadPersonasBus=65 #personas
cantidadParadasEnRuta=20 #cantidad paradas
primeraSalida = 15 # minutos que tarada en salir la primera vez
personasTransportadas=[0,0,0,0,0,0,0]
pasajerosActual=[0,0,0,0,0,0,0]

In [5]:
def subenAlBus():
  return random.randint(3,7)
def bajandDelBus():
  return random.randint(5,7)

In [6]:
def principal(env,bus, posicion):
  primera= True
  almuerzo=True
  cantidad=0
  while True:
    cantidad+=1
    if primera:
      primera=False
      yield env.timeout(primeraSalida*posicion)  

    print(env.now)
    print("------------------------")
    # print("=================>>>>>>>>>>>>>>>>>> bus",posicion,"Cantidada",cantidad)

    recorriendoRuta = tiempoRuta
    while recorriendoRuta:
      for i in range(20):
        if i == 0: 
          suben=7 #subenAlBus()
          personasTransportadas[posicion]+=suben
          pasajerosActual[posicion]= suben # cantidad de personas que suben al bus en la primera parada
          print("suben ", suben, "pasajeros y transportan ", pasajerosActual[posicion], "el bus",posicion)
          recorriendoRuta-=4.5  #tiempo que tarda en llegar en cada parada
        elif i==19:
          print("bajan todos los pasajeros ", "del bus",posicion)
          pasajerosActual[posicion]= 0 # bajan todos los pasajeros del bus en la ultima parada
          recorriendoRuta-=4.5 #tiempo que tarda en llegar en cada parada
          yield env.timeout(descansoFinalizarRecorriedo) #descanza para comenzar el otro recorrido
        else:
          suben=subenAlBus()
          while (pasajerosActual[posicion]+suben)>bus.capacity:
            suben=subenAlBus()
          personasTransportadas[posicion]+=suben
          pasajerosActual[posicion]+= suben
          print("suben ", suben, "pasajeros y transportan ", pasajerosActual[posicion], "el bus",posicion)
          bajan= bajandDelBus()
          if (pasajerosActual[posicion]-bajan)<0: # si se bajan y no queda el bus vacio no toma atencion
            bajan= pasajerosActual[posicion]
          pasajerosActual[posicion]-=bajan #menora a las personas que se bajan del bus
          print("bajan ", bajan, "pasajeros y transportan ", pasajerosActual[posicion], "el bus",posicion)
          recorriendoRuta-=4.5 #tiempo que tarda en llegar en cada parada

        #print(recorriendoRuta)
        yield env.timeout(4.5) #tiempo que tarda en llegar en cada parada

      if env.now >= 360 and almuerzo:
        almuerzo=False  
        print("descanzo del almuerzo")
        yield env.timeout(horaAlmuerzo)
      
  

In [7]:
if __name__ == '__main__':
  #bajandDelBus()
  Buses= collections.namedtuple('buses', 'bus1,bus2,bus3,bus4,bus5,bus6,bus7')
  env = simpy.Environment()
  bus1=simpy.Resource(env,capacity=cantidadPersonasBus)
  bus2=simpy.Resource(env,capacity=cantidadPersonasBus)
  bus3=simpy.Resource(env,capacity=cantidadPersonasBus)
  bus4=simpy.Resource(env,capacity=cantidadPersonasBus)
  bus5=simpy.Resource(env,capacity=cantidadPersonasBus)
  bus6=simpy.Resource(env,capacity=cantidadPersonasBus)
  bus7=simpy.Resource(env,capacity=cantidadPersonasBus)
  buses=Buses(bus1,bus2,bus3,bus4,bus5,bus6,bus7)
 
  for i in range(len(buses)):
    env.process(principal(env,buses[i],i))
  #env.process(principal(env,buses[2],2))
  env.run(until=tiempoSimulacion)


0
------------------------
suben  7 pasajeros y transportan  7 el bus 0
suben  5 pasajeros y transportan  12 el bus 0
bajan  7 pasajeros y transportan  5 el bus 0
suben  5 pasajeros y transportan  10 el bus 0
bajan  7 pasajeros y transportan  3 el bus 0
suben  4 pasajeros y transportan  7 el bus 0
bajan  5 pasajeros y transportan  2 el bus 0
15
------------------------
suben  7 pasajeros y transportan  7 el bus 1
suben  3 pasajeros y transportan  5 el bus 0
bajan  5 pasajeros y transportan  0 el bus 0
suben  6 pasajeros y transportan  13 el bus 1
bajan  6 pasajeros y transportan  7 el bus 1
suben  3 pasajeros y transportan  3 el bus 0
bajan  3 pasajeros y transportan  0 el bus 0
suben  5 pasajeros y transportan  12 el bus 1
bajan  7 pasajeros y transportan  5 el bus 1
suben  5 pasajeros y transportan  5 el bus 0
bajan  5 pasajeros y transportan  0 el bus 0
suben  7 pasajeros y transportan  12 el bus 1
bajan  6 pasajeros y transportan  6 el bus 1
30
------------------------
suben  7 pas

suben  3 pasajeros y transportan  3 el bus 6
bajan  3 pasajeros y transportan  0 el bus 6
suben  7 pasajeros y transportan  10 el bus 2
bajan  5 pasajeros y transportan  5 el bus 2
suben  3 pasajeros y transportan  7 el bus 3
bajan  6 pasajeros y transportan  1 el bus 3
suben  4 pasajeros y transportan  4 el bus 0
bajan  4 pasajeros y transportan  0 el bus 0
suben  6 pasajeros y transportan  6 el bus 1
bajan  6 pasajeros y transportan  0 el bus 1
bajan todos los pasajeros  del bus 6
suben  6 pasajeros y transportan  11 el bus 2
bajan  6 pasajeros y transportan  5 el bus 2
suben  3 pasajeros y transportan  4 el bus 3
bajan  4 pasajeros y transportan  0 el bus 3
suben  3 pasajeros y transportan  3 el bus 0
bajan  3 pasajeros y transportan  0 el bus 0
180.0
------------------------
suben  7 pasajeros y transportan  7 el bus 4
suben  4 pasajeros y transportan  4 el bus 1
bajan  4 pasajeros y transportan  0 el bus 1
suben  4 pasajeros y transportan  9 el bus 2
bajan  7 pasajeros y transport

bajan  7 pasajeros y transportan  3 el bus 5
suben  6 pasajeros y transportan  7 el bus 2
bajan  6 pasajeros y transportan  1 el bus 2
suben  4 pasajeros y transportan  6 el bus 3
bajan  6 pasajeros y transportan  0 el bus 3
suben  6 pasajeros y transportan  7 el bus 0
bajan  7 pasajeros y transportan  0 el bus 0
suben  3 pasajeros y transportan  7 el bus 4
bajan  5 pasajeros y transportan  2 el bus 4
suben  4 pasajeros y transportan  4 el bus 1
bajan  4 pasajeros y transportan  0 el bus 1
suben  5 pasajeros y transportan  8 el bus 5
bajan  7 pasajeros y transportan  1 el bus 5
suben  6 pasajeros y transportan  7 el bus 2
bajan  7 pasajeros y transportan  0 el bus 2
suben  5 pasajeros y transportan  5 el bus 3
bajan  5 pasajeros y transportan  0 el bus 3
bajan todos los pasajeros  del bus 0
suben  6 pasajeros y transportan  8 el bus 4
bajan  7 pasajeros y transportan  1 el bus 4
suben  3 pasajeros y transportan  3 el bus 1
bajan  3 pasajeros y transportan  0 el bus 1
suben  7 pasajeros

suben  7 pasajeros y transportan  7 el bus 6
bajan  7 pasajeros y transportan  0 el bus 6
suben  6 pasajeros y transportan  6 el bus 3
bajan  6 pasajeros y transportan  0 el bus 3
suben  4 pasajeros y transportan  4 el bus 4
bajan  4 pasajeros y transportan  0 el bus 4
suben  5 pasajeros y transportan  8 el bus 5
bajan  6 pasajeros y transportan  2 el bus 5
bajan todos los pasajeros  del bus 2
suben  7 pasajeros y transportan  7 el bus 6
bajan  7 pasajeros y transportan  0 el bus 6
suben  3 pasajeros y transportan  3 el bus 3
bajan  3 pasajeros y transportan  0 el bus 3
suben  3 pasajeros y transportan  3 el bus 4
bajan  3 pasajeros y transportan  0 el bus 4
540.0
------------------------
suben  7 pasajeros y transportan  7 el bus 0
suben  7 pasajeros y transportan  9 el bus 5
bajan  6 pasajeros y transportan  3 el bus 5
suben  6 pasajeros y transportan  6 el bus 6
bajan  6 pasajeros y transportan  0 el bus 6
suben  3 pasajeros y transportan  3 el bus 3
bajan  3 pasajeros y transportan

suben  5 pasajeros y transportan  13 el bus 2
bajan  6 pasajeros y transportan  7 el bus 2
suben  6 pasajeros y transportan  6 el bus 0
bajan  6 pasajeros y transportan  0 el bus 0
bajan todos los pasajeros  del bus 5
suben  3 pasajeros y transportan  3 el bus 1
bajan  3 pasajeros y transportan  0 el bus 1
suben  6 pasajeros y transportan  6 el bus 6
bajan  6 pasajeros y transportan  0 el bus 6
suben  5 pasajeros y transportan  12 el bus 2
bajan  5 pasajeros y transportan  7 el bus 2
705.0
------------------------
suben  7 pasajeros y transportan  7 el bus 3
suben  6 pasajeros y transportan  6 el bus 0
bajan  6 pasajeros y transportan  0 el bus 0
suben  3 pasajeros y transportan  3 el bus 1
bajan  3 pasajeros y transportan  0 el bus 1
suben  3 pasajeros y transportan  3 el bus 6
bajan  3 pasajeros y transportan  0 el bus 6
suben  3 pasajeros y transportan  10 el bus 2
bajan  7 pasajeros y transportan  3 el bus 2
suben  7 pasajeros y transportan  14 el bus 3
bajan  5 pasajeros y transpo

bajan  7 pasajeros y transportan  2 el bus 1
suben  6 pasajeros y transportan  9 el bus 5
bajan  5 pasajeros y transportan  4 el bus 5
suben  6 pasajeros y transportan  11 el bus 2
bajan  5 pasajeros y transportan  6 el bus 2
suben  4 pasajeros y transportan  8 el bus 3
bajan  6 pasajeros y transportan  2 el bus 3
bajan todos los pasajeros  del bus 0
suben  7 pasajeros y transportan  18 el bus 4
bajan  6 pasajeros y transportan  12 el bus 4
suben  7 pasajeros y transportan  9 el bus 1
bajan  5 pasajeros y transportan  4 el bus 1
suben  7 pasajeros y transportan  11 el bus 5
bajan  6 pasajeros y transportan  5 el bus 5
suben  5 pasajeros y transportan  11 el bus 2
bajan  7 pasajeros y transportan  4 el bus 2
870.0
------------------------
suben  7 pasajeros y transportan  7 el bus 6
suben  7 pasajeros y transportan  9 el bus 3
bajan  6 pasajeros y transportan  3 el bus 3
suben  6 pasajeros y transportan  18 el bus 4
bajan  7 pasajeros y transportan  11 el bus 4
suben  5 pasajeros y tran